In [ ]:
from selenium import webdriver
import time
import datetime as dt
from selenium.common.exceptions import NoSuchElementException
import re

import pandas as pd

#파일 경로
save_path = 'c:/cr/app/'
driver_path = 'c:/cd/chromedriver'

# 파일형식
save_type = 'csv'

#헤더 설정
user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) " +\
"AppleWebKit/537.36 (KHTML, like Gecko) " + \
"Chrome/37.0.2062.94 Safari/537.36"
headers = {"User-Agent": user_agent}

#시간 설정
today = dt.datetime.today()
today = today.strftime('%y%m%d')

#접속 URL
#게임: 최고매출
url = 'https://play.google.com/store/apps/category/GAME/collection/topgrossing'
#찾고싶은 앱의 순위
rank = 2
#찾고싶은 페이지 수 (기본 120개, 반복 1회당 120여개 추가) ex. 1일때 240개, 2일때 360개, 5일때 840, 10일때 1680
repeat = 10

In [ ]:
browser = webdriver.Chrome(driver_path)
browser.implicitly_wait(3)
browser.get(url)
#body-content > div.outer-container > div > div.main-content > div > div > div > div.id-card-list.card-list.two-cards > div:nth-child(1)
#body-content > div.outer-container > div > div.main-content > div > div > div > div.id-card-list.card-list.two-cards > div:nth-child(2)

In [ ]:
class ARcrawler:
    def __init__(self, url, rank, repeat):
        self.url = url
        self.rank = rank
        self.repeat = repeat
    
    
    def main(browser):
        ARcrawler.click_app(browser, rank)
        ARcrawler.more_pages(browser, repeat)
        results = ARcrawler.get_reviews(browser)
    
    
    def click_app(browser, rank=1, order=1):
        # rank 순위의 앱 클릭
        browser.find_element_by_xpath('//*[@id="body-content"]/div/div/div[1]/div/div/div/div[2]/div[{}]'.format(rank)).click()
        browser.implicitly_wait(3)
        #리뷰 모두보기 클릭
        browser.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div/div[6]/div/content/span').click()
        #리뷰를 원하는 방식으로 정렬. 기본은 최신. (1. 최신 2. 높은 평점순 3. 유용도순)
        browser.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div/div[2]/c-wiz/div/div/div[1]/div[2]/span').click()
        browser.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div/div[2]/c-wiz/div/div/div[2]/div[{}]'.format(order)).click()
    
    
    def page_scroll(browser):
        SCROLL_PAUSE_TIME = 1

        # Get scroll height
        last_height = browser.execute_script("return document.body.scrollHeight")
        print('*---자동 스크롤 시작---*')
        while True:
            # Scroll down to bottom
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print('*---자동 스크롤 종료---*')
                break
            last_height = new_height
            
    
    def more_pages(browser, repeat=1):
        progress = 0
        for i in range(repeat):
            ARcrawler.page_scroll(browser)
            
            try:
                browser.find_element_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div.Ir7OJc > div > content > span').click()
                progress += 1
                print('*---{}번째 추가 페이지 로딩중---*'.format(progress))
                browser.implicitly_wait(2)
            
            except NoSuchElementException:
                print('*---불러올 페이지가 없습니다---*')
                break
            
            
    def get_reviews(browser):
        print('*---데이터 수집 시작---*')
        ids = browser.find_elements_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div > div > div > div.QV3svb.GVFJbb > div.H9roif.Boieuf > div.GVFJbb.jBjpkf > span')
        rates = browser.find_elements_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div > div > div > div.QV3svb.GVFJbb > div.H9roif.Boieuf > div.GVFJbb.jBjpkf > div > span.qC3s2c > div > div')
        dates = browser.find_elements_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div > div > div > div.QV3svb.GVFJbb > div.H9roif.Boieuf > div.GVFJbb.jBjpkf > div > span.oldIDd')
        reviews = browser.find_elements_by_css_selector('div.JNury.Ekdcne > div > div > div > div > div > div > div > div.QV3svb.GVFJbb > div.Z8UXhc > span')
        print('*---데이터 수집 종료---*')
        
        special_chars_remover = re.compile("[^\w'|_]")
        date_chars_remover = re.compile('(년 |월 |일)')
        
        results = []
        
        n = len(ids)
        print('*---데이터 정제 시작---*')
        for i in range(n):
            if i > 0 and i%200 == 0:
                print('*---{}번째데이터 정제중---*'.format(i))
            
            refined_id = special_chars_remover.sub(' ', ids[i].text)
            refined_rate = rates[i].get_attribute('aria-label').replace('별표 5개 만점에 ', '').replace('개를 받았습니다.', '')
            refined_date = date_chars_remover.sub('.', dates[i].text)
            refined_review = special_chars_remover.sub(' ', reviews[i*2].text)
            results.append([refined_id, refined_rate, refined_date, refined_review])
            
            if i > 0 and i%500 == 0:
                index = str(i)+'of'+str(n)
                ARcrawler.save(index, results)
                print('*---중간 저장 완료({}) ---*'.format(index))
        
        print('*---크롤링 완료---*')
        ARcrawler.save('final', results)
        print('*---저장 완료---*')
        
        return(results)
 

    def save(index, results):
        txt_path = save_path + '{}_app_reviews_{}.txt'.format(index, today)
        with open(txt_path, 'w+', encoding = 'utf-8')as f:
            for result in results:
                f.write(','.join(result))
                f.write('\n')

In [ ]:
ARcrawler.main(browser)